In [1]:
from google import generativeai as genai
genai.configure(api_key='******************')

C:\Anaconda\envs\genai\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def get_completion_from_messages(messages, model="gemini-2.0-flash-lite",
                                temp=0, max_tokens=500):
    model= genai.GenerativeModel(model)

    response = model.generate_content(messages[-1]['content'], generation_config ={"temperature":temp, 
                                                                                   "max_output_tokens":max_tokens})
    return response.text
    

In [3]:
final_respone_to_customer=f"""
Okay! I can help with that.

**SmartX ProPhone:** It has a 6.1-inch display, 128GB storage, a 12MP dual camera, and 5G. It's $899.99.

**FotoSnap DSLR Camera:** It has a 24.2MP sensor, 1080p video, a 3-inch LCD, and interchangeable lenses. It's $599.99.

**Regarding TVs, we have a few options:**

*   **CineView 4K TV:** 55-inch, 4K resolution, HDR, and Smart TV for $599.99.
*   **CineView 8K TV:** 65-inch, 8K resolution, HDR, and Smart TV for $2999.99.
*   **CineView OLED TV:** 55-inch, 4K resolution, HDR, and Smart TV for $1499.99.

Do any of those TVs sound interesting?
"""

In [4]:
moderation_prompt=f"""
You are a content moderator. Analyze the following text for harmful content, hate speech, harassment, or unsafe topics.
Respond with a single letter:
Y - if it contains harmful content
N - if it is safe.

text: '''{final_respone_to_customer}
"""

moderation_response= get_completion_from_messages([{"role":"user", "content":moderation_prompt}])

print("Moderation Check:", moderation_response)

Moderation Check: N



In [6]:
system_message=f"""
You are an assistant that evaluates whether \
customer service agent responses sufficiently \
answer customer questions, and also validates that \
all the facts the assistant cites from the product \
information are correct.
The product information and user and customer \
service agent messages will be delimited by \
3 backticks, i.e. \`\`\`.
Respond with a Y or N character, with no punctuation:
Y - if the output sufficiently answers the question \
AND the response correctly uses product information
N - otherwise

Output a single letter only.
"""

customer_message=f"""
tell me about the smartx pro phone and \
the fotosnap camera, the dslr one. \
Also tell me about your tvs"""

product_information = """{ "name": "SmartX ProPhone", "category": "Smartphones and Accessories", "brand": "SmartX", "model_number": "SX-PP10", "warranty": "1 year", "rating": 4.6, "features": [ "6.1-inch display", "128GB storage", "12MP dual camera", "5G" ], "description": "A powerful smartphone with advanced camera features.", "price": 899.99 } { "name": "FotoSnap DSLR Camera", "category": "Cameras and Camcorders", "brand": "FotoSnap", "model_number": "FS-DSLR200", "warranty": "1 year", "rating": 4.7, "features": [ "24.2MP sensor", "1080p video", "3-inch LCD", "Interchangeable lenses" ], "description": "Capture stunning photos and videos with this versatile DSLR camera.", "price": 599.99 } """

q_a_pair=f"""
Customer message:
{customer_message}
Product information:
{product_information}
Agent response:
{final_respone_to_customer}
Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""

messages=[
    {"role":"system","content":system_message},
    {"role":"user","content":q_a_pair}]

response=get_completion_from_messages(messages, max_tokens=1)
print(response)

Y


In [7]:
#wrong Response for Checking

another_response="""life is like a box of chocolates"""

q_a_pair=f"""
Customer message:
{customer_message}
Product information:
{product_information}
Agent response:
{another_response}
Does the response use the retrieved information correctly?
Does the response sufficiently answer the question?

Output Y or N
"""

messages=[
    {"role":"system","content":system_message},
    {"role":"user","content":q_a_pair}]

response=get_completion_from_messages(messages, max_tokens=1)
print(response)

N
